In [2]:
import numpy as np
import numpy.matlib
#import non_rigid_registration
#import normPts
#import similarity_fitting

In [3]:
import scipy.io
data = scipy.io.loadmat('all_you_need.mat') 


In [4]:
vs=data['VS']
fs=data['FS']
vt=data['VT']
ft=data['FT']
vs2 = data['VS2']
fs2 = data['FS2']
marker=data['marker']

In [6]:
print("VS.shape",vs.shape)
print("FS.shape",fs.shape)
print("VT.shape",vt.shape)
print("FT.shape",ft.shape)
print("marker.shape",marker.shape)

VS.shape (29299, 3)
FS.shape (57836, 3)
VT.shape (15941, 3)
FT.shape (31620, 3)
marker.shape (38, 2)


In [7]:
corres=scipy.io.loadmat('corres.mat')

In [8]:
corres=corres['corres']

In [9]:
name=[]
for i in range(len(marker)):
    name.append(i+1)

In [10]:
S_factor=np.zeros(3)
T_factor=np.zeros(3)
S_size=vs.shape[0]

In [11]:
tmean=0
tstd=2**0.5

In [12]:
import numpy as np
import numpy.matlib
def normPts(x,m,s):
    n=x.shape[0]
    k=x.shape[1]
    T=np.eye(k+1)
    
    mu=np.mean(x,0)
    T[0:k,k]=0-mu
    temp=np.power(x-np.matlib.repmat(mu,n,1),2)
    temp=np.power(temp,0.5)
    mean_distance=np.mean(np.sum(temp,1))
    
    scale=s/mean_distance
    T=scale*T
    T[k,k]=1

    temp=np.ones(len(x))
    temp = temp[:,np.newaxis]
    
    x=np.hstack((x,temp))
    x=np.transpose(np.matmul(T,np.transpose(x)))
    x=x[:,0:k]
    return x

In [13]:
vs=normPts(vs,tmean,tstd)
vt=normPts(vt,tmean,tstd)

In [14]:
import similarity_fitting as s_f

In [15]:
R, t, s, res=s_f.similarity_fitting(vt[list(marker[:,1]-1),:],vs[list(marker[:,0]-1),:])

In [16]:
vt=np.matmul(vt,np.transpose(s*R))+np.matlib.repmat(t,len(vt),1)

In [17]:
import numpy.matlib
def v4_normal(Vert,Face):
    f1=Face[:,0]
    f2=Face[:,1]
    f3=Face[:,2]
    
    e1=Vert[list(f2-1),:]-Vert[list(f1-1),:]
    e2=Vert[list(f3-1),:]-Vert[list(f1-1),:]

    #e2=
    c=np.matlib.cross(e1,e2,1)
    c_norm=(np.power(c[:,0:1],2)+np.power(c[:,1:2],2)+np.power(c[:,2:3],2))**0.5
    c_norm[c_norm==0]=1
    N=c/np.matlib.repmat(c_norm,1,3)
    v4=Vert[list(f1-1),:]+N
    V=np.vstack((Vert,v4))
    F=Face
    temp=len(Vert)+numpy.arange(1,len(F)+1)
    temp=temp[:,np.newaxis]
    F=np.hstack((F,temp))
    T=[]
    for i in range(0,len(F)):
        x=V[F[i,1]-1,]-V[F[i,0]-1,]
        y=V[F[i,2]-1,]-V[F[i,0]-1,]
        z=V[F[i,3]-1,]-V[F[i,0]-1,]
        t=np.array([x,y,z])
        T.append(np.transpose(t))
    return T,N,V,F

In [18]:
ts,ns,vs4,fs4=v4_normal(vs,fs)
tt,nt,vt4,ft4=v4_normal(vt,ft)

In [25]:
Adj_idx=np.zeros((len(fs),3),dtype=u'int64')

In [24]:
def Adj(i):
    for j in range(3):
        temp1=np.sum(fs==fs[i,j],1) 
        temp2=np.sum(fs==fs[i,(j+1)%3],1)
        idx=np.nonzero(temp1 & temp2)
        temp=np.array([i]*len(idx[0]))
        if np.sum(idx!=temp):
            Adj_idx[i,j]=idx[0][np.nonzero(((idx!=temp)+0)[0])]+1


In [29]:
from joblib import Parallel,delayed
Parallel(n_jobs=1000,require='sharedmem')(delayed(Adj)(i) for i in range(len(fs)))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [30]:
Adj_idx


array([[    2,     0,    19],
       [    3,     0,     1],
       [    4,     2,    21],
       ...,
       [57835, 57820, 57833],
       [57836, 57834,     0],
       [    0, 57822, 57835]], dtype=int64)

In [20]:
def build_elementary_cell(ts,l):
    E=[]
    for i in range(l):
        temp2=np.linalg.inv(ts[i])
        temp1=-np.sum(temp2,0)
        E.append(np.transpose(np.vstack((temp1,temp2))))
    return E
    


In [21]:
E=build_elementary_cell(ts,len(fs))

In [22]:
E[999]

array([[ 19.69904547,  28.59380862, -48.69017342,   0.39731933],
       [-57.45826044,  -3.45765642,  60.43448538,   0.48143148],
       [ 24.10912908, -12.41110102, -12.47928415,   0.78125609]])

In [23]:
n_adj=Adj_idx.size
n_adj

173508

In [24]:
len_col=np.max(fs4)#需要确认
len_col

87135

In [25]:
ws=1.0
wi=0.01
I1=np.zeros([9*n_adj*4,3])
I2=np.zeros([9*n_adj*4,3])
I3=np.zeros([9*len(fs4)*4,3])
C1=np.zeros([9*n_adj,1])
C2=wi*np.matlib.reshape(np.transpose(np.matlib.repmat(np.matlib.reshape(np.eye(3),9,1),len(fs4),1)),9*len(fs4),1)

In [26]:
C2.shape

(520524,)

In [26]:
reverseStr=[]

In [27]:
for i in range(len(fs4)):
    for j in range(3):
        if Adj_idx[i,j]:
            constid=np.zeros([2,4])
            for k in range(3):
                if np.sum(marker[:,0:1]==fs4[i,k]):
                    constid[0,k]=(k+1)*np.sum(marker[:,0:1]==fs4[i,k])
                if np.sum(marker[:,0:1]==fs4[Adj_idx[i,j]-1,k]):
                    constid[1,k]=(k+1)*np.sum(marker[:,0:1]==fs4[Adj_idx[i,j]-1,k])
            U1=fs4[i]
            U2=fs4[int(Adj_idx[i,j])-1]
            for k in range(3):
                row=np.matlib.repmat(np.array([1,2,3])+i*3*3*3+j*3*3+k*3,4,1)
                col1=np.matlib.repmat((U1-1)*3+k+1,3,1)
                col1=np.transpose(col1)
                val1=ws*np.transpose(E[i])
                if np.sum(constid[0]):
                    C1[np.array([1,2,3])+i*3*3*3+j*3*3+k*3-1]=C1[np.array([1,2,3])+i*3*3*3+j*3*3+k*3-1]-np.transpose(val1[np.nonzero(constid[0]>0)])*np.transpose(vt4[marker[np.nonzero((marker==U1[constid[0]>0])+0),1][0]-1,k])
                val1[constid[0]>0]=0
                col2=np.transpose(np.matlib.repmat((U2-1)*3+k+1,3,1))
                val2=np.transpose(-ws*E[Adj_idx[i,j]-1])
                if np.sum(constid[1]):
                    C1[np.array([1,2,3])+i*3*3*3+j*3*3+k*3-1]=C1[np.array([1,2,3])+i*3*3*3+j*3*3+k*3-1]-np.transpose(val2[np.nonzero(constid[1]>0)])*np.transpose(vt4[marker[np.nonzero((marker==U2[constid[1]>0])+0),1][0]-1,k])
                a=np.matlib.reshape(row,12,1)
                b=np.matlib.reshape(col1,12,1)
                c=np.matlib.reshape(val1,12,1)
                I1[np.arange(0,12)+i*3*3*3*4+j*3*3*4+k*3*4]=np.transpose(np.vstack((a,b,c)))
                b=np.matlib.reshape(col2,12,1)
                c=np.matlib.reshape(val2,12,1)
                I2[np.arange(0,12)+i*3*3*3*4+j*3*3*4+k*3*4]=np.transpose(np.vstack((a,b,c)))              

In [29]:
I1=I1[I1[:,0]>0]
I2=I2[I2[:,0]>0]



In [30]:
max(I1[:,0])

1561572.0

In [33]:
from scipy.sparse import csr_matrix


In [68]:
M1=csr_matrix((I1[:,2],(I1[:,0]-1,I1[:,1]-1)),shape=(9*n_adj,9*n_adj+int(np.max(I3[:,0]))))
M2=csr_matrix((I2[:,2],(I2[:,0]-1,I2[:,1]-1)),shape=(9*n_adj,9*n_adj+int(np.max(I3[:,0]))))

In [69]:
M1

<1561572x2082096 sparse matrix of type '<class 'numpy.float64'>'
	with 6218856 stored elements in Compressed Sparse Row format>

In [70]:
M2

<1561572x2082096 sparse matrix of type '<class 'numpy.float64'>'
	with 6218856 stored elements in Compressed Sparse Row format>

In [71]:
M3=M1+M2

In [72]:
M3

<1561572x2082096 sparse matrix of type '<class 'numpy.float64'>'
	with 9326574 stored elements in Compressed Sparse Row format>

In [38]:
for i in range(len(fs4)):
    U1=fs4[i]
    for k in range(3):
        row=np.matlib.repmat(np.array([1,2,3])+i*3*3+k*3,4,1)
        col1=np.matlib.repmat((U1-1)*3+k+1,3,1)
        col1=np.transpose(col1)
        val1=wi*np.transpose(E[i])
        a=np.matlib.reshape(row,12,1)
        b=np.matlib.reshape(col1,12,1)
        c=np.matlib.reshape(val1,12,1)
        I3[np.arange(0,12)+i*3*3*4+k*3*4]=np.transpose(np.vstack((a,b,c)))
        
        

In [66]:
M4=csr_matrix((I3[:,2],(I3[:,0]-1,I3[:,1]-1)),(int(np.max(I3[:,0])),9*n_adj+int(np.max(I3[:,0]))))

In [67]:
M4

<520524x2082096 sparse matrix of type '<class 'numpy.float64'>'
	with 2082096 stored elements in Compressed Sparse Row format>

<2082096x2082096 sparse matrix of type '<class 'numpy.float64'>'
	with 11408670 stored elements in Compressed Sparse Row format>

In [46]:
C=np.hstack((np.squeeze(C1),C2))

In [60]:
C.shape

(2082096,)

In [48]:
import scipy.sparse

In [73]:
M=scipy.sparse.vstack((M3,M4))

In [74]:
M

<2082096x2082096 sparse matrix of type '<class 'numpy.float64'>'
	with 11408670 stored elements in Compressed Sparse Row format>

In [63]:
import scipy.sparse.linalg

ModuleNotFoundError: No module named 'scipy.sparse.linalg.spsolve'

In [75]:
t1res=scipy.sparse.linalg.spsolve(M,C,use_umfpack=True)

RuntimeError: failed to factorize matrix at line 110 in file scipy\sparse\linalg\dsolve\SuperLU\SRC\dsnode_bmod.c


In [65]:
from scikits.umfpack import spsolve

ModuleNotFoundError: No module named '__umfpack'

In [ ]:
scipy.sparse.linalg.spsolve

In [39]:
result_x = scipy.sparse.linalg.lsqr(M,C,show=True,iter_lim = 1000)

 
LSQR            Least-squares solution of  Ax = b
The matrix A has 2.0821e+06 rows  and   261405 cols
damp = 0.00000000000000e+00   calc_var =        0
atol = 1.00e-08                 conlim = 1.00e+08
btol = 1.00e-08               iter_lim =     1000
 
   Itn      x[0]       r1norm     r2norm   Compatible    LS      Norm A   Cond A
     0  0.00000e+00   1.963e+03  1.963e+03    1.0e+00  1.8e-01
     1 -4.78574e-10   1.501e+03  1.501e+03    7.6e-01  5.6e-01   5.6e+02  1.0e+00
     2 -1.80332e-09   1.286e+03  1.286e+03    6.6e-01  2.5e-01   9.0e+02  2.4e+00
     3 -4.53668e-09   1.178e+03  1.178e+03    6.0e-01  1.8e-01   1.1e+03  3.9e+00
     4 -9.47493e-09   1.098e+03  1.098e+03    5.6e-01  1.3e-01   1.3e+03  5.8e+00
     5 -1.71041e-08   1.036e+03  1.036e+03    5.3e-01  1.2e-01   1.5e+03  7.8e+00
     6 -2.79494e-08   9.822e+02  9.822e+02    5.0e-01  1.0e-01   1.7e+03  1.0e+01
     7 -4.18721e-08   9.353e+02  9.353e+02    4.8e-01  9.3e-02   1.9e+03  1.3e+01
     8 -5.85043e-08   8.93

In [48]:
VSP1 = result_x[0]

In [54]:
VSP1=np.matlib.reshape(VSP1,[3,int(len(VSP1)/3)])
VSP1=np.transpose(VSP1)
VSP1=VSP1[0:S_size,:]

In [59]:
VS_Reg = VSP1
VT_Reg = vt

In [62]:
from  build_correspondence import build_correspondence
corres = build_correspondence(VS_Reg, fs, VT_Reg, ft, 10, 0.05)

In [104]:

import numpy as np
ts,ns,vs4,fs4=v4_normal(vs,fs)
ts2,ns2,vs42,fs42=v4_normal(vs2,fs2)

In [162]:
ft4

array([[15941,     1,     2, 15942],
       [15941,     2,     3, 15943],
       [    4,     5,     6, 15944],
       ...,
       [13438, 12244, 13439, 47559],
       [12244,  6490, 12609, 47560],
       [12244, 12609, 13439, 47561]])

In [37]:

import numpy as np

def deformation_transfer(vs,fs,vt,ft,vs2,fs2,corres):
    lenfs=len(fs)
    lenft=len(ft)
    SD=[]
    ts,ns,vs4,fs4=v4_normal(vs,fs)
    ts2,ns2,vs42,fs42=v4_normal(vs2,fs2)
    tt,nt,vt4,ft4=v4_normal(vt,ft)

    for i in range(lenfs):
        SD.append(np.matmul(ts2[i],np.linalg.inv(ts[i])))#矩阵除法
    
    E=build_elementary_cell(tt,lenft)

    n_corres = sum([len(tmp) for tmp in corres])
    n_non_corres = np.sum([list(tmp)==[] for tmp in corres])
    print(n_corres)
    print(n_non_corres)
    I=np.zeros([9*(n_corres+n_non_corres)*4,3])
    C=np.zeros([9*(n_corres+n_non_corres),1])

    print('Transfer deformation')

    offset=0
    offset2=0

    for i in range(lenft):
        lencor=len(corres[i])
        cor=corres[i]
        U=ft4[i]
        if lencor:
            for j in range(lencor):
                for k in range(3):
                    row=np.matlib.repmat(np.array([1,2,3])+offset+j*3*3+k*3,4,1)
                    col1=np.matlib.repmat((U-1)*3+k,3,1)
                    col1=np.transpose(col1)
                    val1=np.transpose(E[i])

                    a=np.matlib.reshape(row,12,1)
                    b=np.matlib.reshape(col1,12,1)
                    c=np.matlib.reshape(val1,12,1)
                    I[np.arange(0,12)+offset2+j*3*3*4+k*3*4]=np.transpose(np.vstack((a,b,c)))

                C[np.arange(0,9)+offset+j*9,0]=np.matlib.reshape(np.transpose(SD[int(cor[j])-1]),9,1)
            offset=offset+3*3*lencor
            offset2=offset2+3*3*lencor*4
        else:
            for k in range(3):
                row=np.matlib.repmat(np.array([1,2,3])+offset+k*3,4,1)
                col1=np.matlib.repmat((U-1)*3+k,3,1)
                col1=np.transpose(col1)
                val1=np.transpose(E[i])
                a=np.matlib.reshape(row,12,1)
                b=np.matlib.reshape(col1,12,1)
                c=np.matlib.reshape(val1,12,1)

                I[np.arange(0,12)+offset2+k*3*4]=np.transpose(np.vstack((a,b,c)))
            C[np.arange(0,9)+offset,0]=np.matlib.reshape(np.eye(3),9,1)
            offset=offset+3*3
            offset2=offset2+3*3*4
    #return I
    M=csr_matrix((I[:,2],(I[:,0]-1,I[:,1]-1)),(int(np.max(I[:,0])),int(np.max(I[:,1]))))
    #M4=csr_matrix((I3[:,2],(I3[:,0]-1,I3[:,1]-1)),(int(np.max(I3[:,0])),int(np.max(I3[:,1]))))
    return M
'''
    x=
    x=np.matlib.reshape(x,3,len(x)/3)
    x=np.transpose(x)
    x=x[0:len(vt),:]

    _,nx=v4_normal(x,ft)

    return x,nx
'''

'\n    x=\n    x=np.matlib.reshape(x,3,len(x)/3)\n    x=np.transpose(x)\n    x=x[0:len(vt),:]\n\n    _,nx=v4_normal(x,ft)\n\n    return x,nx\n'

In [103]:
ts,ns,vs4,fs4=v4_normal(vs,fs)

In [104]:
ts

[array([[-0.02171   , -0.154346  , -0.11726636],
        [ 0.081997  , -0.014164  ,  0.43748713],
        [-0.043092  , -0.013351  ,  0.89154563]]),
 array([[ 0.129914  , -0.02171   , -0.10989307],
        [ 0.097168  ,  0.081997  ,  0.43943065],
        [-0.03188   , -0.043092  ,  0.89152914]]),
 array([[-0.021044  , -0.150958  , -0.11063494],
        [ 0.081785  , -0.015383  ,  0.4402584 ],
        [-0.043023  , -0.011143  ,  0.89102887]]),
 array([[ 0.127047  , -0.021044  , -0.11405911],
        [ 0.097781  ,  0.081785  ,  0.43937865],
        [-0.031954  , -0.043023  ,  0.89103138]]),
 array([[-0.020198  , -0.147245  , -0.10959269],
        [ 0.081734  , -0.016047  ,  0.4345335 ],
        [-0.042205  , -0.010251  ,  0.89396313]]),
 array([[ 0.124014  , -0.020198  , -0.10427772],
        [ 0.09772   ,  0.081734  ,  0.43584543],
        [-0.033177  , -0.042205  ,  0.89396024]]),
 array([[-0.019678  , -0.143692  , -0.09773972],
        [ 0.081471  , -0.016249  ,  0.42893128],
        

In [221]:
lenfs=len(fs)
lenft=len(ft)
SD=[]
ts,ns,vs4,fs4=v4_normal(vs,fs)
ts2,ns2,vs42,fs42=v4_normal(vs2,fs2)
tt,nt,vt4,ft4=v4_normal(vt,ft)

for i in range(lenfs):
    SD.append(np.matmul(ts2[i],np.linalg.inv(ts[i])))#矩阵除法
    
E=build_elementary_cell(tt,lenft)

n_corres = 0
n_non_corres=0
for i in range(len(corres)):
    if len(corres[i][0]):
        n_corres += len(corres[i][0][0])
    else:
        n_non_corres+=1

print(n_corres)
print(n_non_corres)
I=np.zeros([9*(n_corres+n_non_corres)*4,3])
C=np.zeros([9*(n_corres+n_non_corres),1])

45551
28832


In [ ]:
corres[804][0]

In [222]:
offset=0
offset2=0

for i in range(lenft):
    if len(corres[i][0]):
        lencor=len(corres[i][0][0])
        cor=corres[i][0][0]
    else:
        lencor=0
        cor=corres[i][0]
    
    U=ft4[i]
    if lencor:
        for j in range(lencor):
            for k in range(3):
                row=np.matlib.repmat(np.array([1,2,3])+offset+j*3*3+k*3,4,1)
                col1=np.matlib.repmat((U-1)*3+k+1,3,1)
                col1=np.transpose(col1)
                val1=np.transpose(E[i])

                a=np.matlib.reshape(row,12,1)
                b=np.matlib.reshape(col1,12,1)
                c=np.matlib.reshape(val1,12,1)
                I[np.arange(0,12)+offset2+j*3*3*4+k*3*4]=np.transpose(np.vstack((a,b,c)))

            C[np.arange(0,9)+offset+j*9,0]=np.matlib.reshape(np.transpose(SD[int(cor[j-1])-1]),9,1)
        offset=offset+3*3*lencor
        offset2=offset2+3*3*lencor*4
    else:
        for k in range(3):
            row=np.matlib.repmat(np.array([1,2,3])+offset+k*3,4,1)
            col1=np.matlib.repmat((U-1)*3+k+1,3,1)
            col1=np.transpose(col1)
            val1=np.transpose(E[i])
            a=np.matlib.reshape(row,12,1)
            b=np.matlib.reshape(col1,12,1)
            c=np.matlib.reshape(val1,12,1)

            I[np.arange(0,12)+offset2+k*3*4]=np.transpose(np.vstack((a,b,c)))
        C[np.arange(0,9)+offset,0]=np.matlib.reshape(np.eye(3),9,1)
        offset=offset+3*3
        offset2=offset2+3*3*4

In [228]:
import scipy.sparse

In [229]:
M=scipy.sparse.csr_matrix((I[:,2],(I[:,0]-1,I[:,1]-1)),(int(np.max(I[:,0])),int(np.max(I[:,1]))))


In [240]:
temp_M=scipy.sparse.csr_matrix.dot(np.transpose(M),M)

In [241]:
temp_C=scipy.sparse.csr_matrix.dot(np.transpose(M),C)

In [244]:
temp_result=scipy.sparse.linalg.spsolve(temp_M,temp_C)

In [247]:
temp_result

array([ 3.30938072, -9.53900969, -8.92399069, ..., -8.2264658 ,
        3.39446126, -4.45912309])

In [261]:
x=temp_result
x=np.reshape(x,[3,int(len(x)/3)],order='F')
x=np.transpose(x)
x=x[0:len(vt),:]

In [262]:
x

array([[  3.30938072,  -9.53900969,  -8.92399069],
       [  2.88501187, -10.3136308 ,  -8.35729428],
       [  3.18431639, -10.2997057 ,  -8.68887068],
       ...,
       [  4.58507342,   2.32231575,  -8.44337755],
       [ -0.10163507,   1.5760197 ,   0.35288089],
       [  3.50849513,  -9.46502328,  -9.28161622]])

In [273]:
_,nx,_,_=v4_normal(x,ft)

In [274]:
nx

array([[ 0.87691095, -0.14520055,  0.45819644],
       [ 0.72500109,  0.18879619,  0.66236653],
       [ 0.2985411 ,  0.68086085,  0.66880619],
       ...,
       [-0.97276701,  0.17699689,  0.14965441],
       [-0.9685802 ,  0.19910399,  0.1490302 ],
       [-0.97339709,  0.17222708,  0.15111567]])

7245

In [146]:
import scipy.sparse as ss